# Loads and Imports

In [3]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [4]:
#built-in
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import sys
import boto3

path = str(Path.home()) + '/rpi-canary-eval/temp'
git_path = str(Path.home()) + '/rpi-canary-eval'
sys.path.append(git_path + '/modules')


#custom
import data_process
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from path import path_dict, col_seq_cc, col_seq_elx, col_seq_cc_all

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

# Download and Import Latent Space data and corresponding Original Data from S3

In [5]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_knn_no_cluster_only_latent/treated0_l.p ../../temp/ldata/treated0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_knn_no_cluster_only_latent/match0_l.p ../../temp/ldata/match0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_knn_no_cluster_only_latent/treated0_o.p ../../temp/odata/treated0_o.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_knn_no_cluster_only_latent/match0_o.p ../../temp/odata/match0_o.p

In [6]:
import pickle
with open(path+'/ldata_knn/match0_l.p', 'rb') as fp:
    controls_sick_latent = pickle.load(fp)
with open(path+'/ldata_knn/treated0_l.p', 'rb') as fp:
    treated_sick_latent = pickle.load(fp)
with open(path+'/odata_knn/match0_o.p', 'rb') as fp:
    controls_sick_original = pickle.load(fp)
with open(path+'/odata_knn/treated0_o.p', 'rb') as fp:
    treated_sick_original = pickle.load(fp)

# Sort and Filter

In [7]:
controls = controls_sick_latent[controls_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated = treated_sick_latent[treated_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID","MYR"])

In [8]:
controls_orig = controls_sick_original[controls_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated_orig = treated_sick_original[treated_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])

## Evaluation Quality Match

## Metric = ASMD

In [9]:
def ASMD_calculate(cols, tf, cf):
    ASMD = pd.DataFrame()
    for col in cols:
        t_mean = np.mean(tf[col].values)
        c_mean = np.mean(cf[col].values)
        t_var = np.var(tf[col].values)
        c_var = np.var(cf[col].values)
        d = np.abs((t_mean-c_mean)/np.sqrt((t_var+c_var)/2))
        ASMD.loc[0, col] = d
    return ASMD

In [10]:
def ASMD_final(cols, cdf, tdf):
    cf = cdf[cols]
    tf = tdf[cols]
    ASMD = ASMD_calculate(cols, tf, cf)
    return ASMD

### Latent Space

In [11]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [12]:
ASMD = ASMD_final(latent_cols, controls, treated)
ASMD

x1        x2        x3   x4   x5        x6   x7        x8   x9       x10       x11  x12  x13  x14       x15  x16
0  1.414214  0.632456  1.897367  0.0  0.0  1.414214  0.0  1.414214  2.0  1.414214  1.414214  0.0  0.0  0.0  1.414214  0.0

### Original Space with George's Features only

In [13]:
george_cols = ['AGE_AT_MIDMONTH', 'MBR_GNDR', 'ALLOW_AMT','CC_TOBACCO',  'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID']

In [14]:
ASMD = ASMD_final(george_cols, controls_orig, treated_orig)
ASMD

AGE_AT_MIDMONTH  MBR_GNDR  ALLOW_AMT  CC_TOBACCO  CC_OBESITY  CC_HYPERTENSION  CC_HYPOTHYROID
0         0.204013  0.521363   0.153812    0.148072    0.476694          0.09225        0.144306

### Original Space with all the features

In [15]:
original_cols = controls_orig.columns.to_list()[2:]

In [16]:
ASMD = ASMD_final(original_cols, controls_orig, treated_orig)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [17]:
ASMD

AGE_AT_MIDMONTH  MBR_GNDR  MEDICAID_LOB  ALLOW_AMT  ALLOW_ER  ALLOW_IP   CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS    CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR    CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER   CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0         0.204013  0.521363      0.747507   0.153812  0.086597  0.080146  0.137169           NaN           0.184716  0.063864   0.022221              0.04284      0.007567   0.060385        0.11082   0.146101    0.106142  0.099692          0.026769     0.054341           0.047823           0.123383              0.021094  0.087993            0.050615       0.017315                 0.037675           0.048605     0.106419         0.097589   0.128234               0.022693     0.057436         0.012203     0.106589           0.040671          0.040695             NaN              0.025295                0.016558                   NaN                0.052635                    0.037035             NaN             NaN          0.030468              NaN           0.052014          0.09225        0.144306      0.190539     0.103583        0.039188      0.113166  0.154103        0.056598     0.285275             0.03759               0.144454                    NaN    0.476694   

   CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER   CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  
0         0.015838       0.006855        0.086837     0.113207            0.081393  0.068098          0.009504                0.156847        0.027042         0.062885   0.005997    0.148072            0.055783          0.016558